### To run this model yourself: go to Run -> Run All Cells in the top left menu bar.

In [ ]:
from epx import Job, ModelConfig, SynthPop

import pandas as pd
import numpy as np
import time

import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go

import plotly.io as pio
from plotly.subplots import make_subplots
import requests

# Use the Epistemix default plotly template
r = requests.get("https://gist.githubusercontent.com/daniel-epistemix/8009ad31ebfa96ac97b7be038c014c0d/raw/320c3b0ca3dfbf7946e49c97254fa65d4753aeac/epx_plotly_theme.json")
if r.status_code == 200:
    pio.templates["epistemix"] = go.layout.Template(r.json())
    pio.templates.default = "epistemix"

# Epstein rebellion model (grid implementation)

This model captures the evolution of civil violence in a population split into regular agents, who are disaffected and willing to rebel if the conditions are right, and cop agents, who try to quell rebellion by jailing those agents.

In this implementation, agents move across a simple grid, making a single move each day. Based on their close neighbors (determined by the vision parameter), they assess if their personal grievance is higher than their risk of being arrested (determined by the ratio of already rebelling agents to cops in the neighborhood).

### Key simulation variables

- simulation grid size: 40x40
- agent density: 0.7
- cop density: 0.05
- government legitimacy: 0.7
- max jail sentence: 15 days
- agent vision: 7
- rebellion threshold: 0.1

## Agent setup

This model uses custom agents generated below, rather than the built in synthetic population.

In [ ]:
neighborhood_size = 40
tot_places = neighborhood_size**2

agent_density = 0.7
cop_density = 0.05

n_regular_agents = int(agent_density*tot_places)
n_cop_agents = int(cop_density*tot_places)

nx, ny = np.meshgrid(np.arange(1,neighborhood_size+1), 
                     np.arange(1,neighborhood_size+1))

place_agents = pd.DataFrame()
place_agents["ID"] = np.arange(1,neighborhood_size**2+1)
place_agents["x_pos"] = nx.ravel()
place_agents["y_pos"] = ny.ravel()
place_agents["agent_type"] = 1

regular_id_int = 10000
regular_agents = pd.DataFrame()
regular_agents["ID"] = np.arange(regular_id_int+1,regular_id_int+n_regular_agents+1)
regular_agents["agent_type"] = 2

cop_id_int = 20000
cop_agents = pd.DataFrame()
cop_agents["ID"] = np.arange(cop_id_int+1,cop_id_int+n_cop_agents+1)
cop_agents["agent_type"] = 3

place_agents.to_csv("place_agents.csv", index=False)
regular_agents.to_csv("regular_agents.csv", index=False)
cop_agents.to_csv("cop_agents.csv", index=False)

## Running the Model

In [ ]:
epstein_rebellion_grid_config = ModelConfig(
             synth_pop=SynthPop("US_2010.v5", ["None"]),
             start_date="2023-01-01",
             end_date="2023-04-01",
         )

epstein_rebellion_grid_job = Job(
    "model/rebellion.fred",
    config=[epstein_rebellion_grid_config],
    key="cl_epstein_rebellion_grid_job",
    fred_version="11.0.1",
    results_dir="/home/epx/cl-results"
)


epstein_rebellion_grid_job.execute()

# the following loop idles while we wait for the simulation job to finish and periodically prints an update
update_count = 0
update_interval = 3
start_time = time.time()
timeout   = 500 # timeout in seconds
idle_time = 20   # time to wait (in seconds) before checking status again
while str(epstein_rebellion_grid_job.status) != 'DONE':
    if str(epstein_rebellion_grid_job.status) == 'ERROR':
        logs = epstein_rebellion_grid_job.status.logs
        log_msg = "; ".join(logs.loc[logs.level == "ERROR"].message.tolist())
        print(f"Job failed with the following error:\n '{log_msg}'")
        break
    if time.time() > start_time + timeout:
        msg = f"Job did not finish within {timeout / 60} minutes."
        raise RuntimeError(msg)
    
    if update_count >= update_interval:
        update_count = 0
        print(f"Job is still processing after {time.time() - start_time:.0f} seconds")
        
    update_count += 1
    
    time.sleep(idle_time)

print(f"Job completed in {time.time() - start_time:.0f} seconds")

str(epstein_rebellion_grid_job.status)

## Visualizing the Results

In [ ]:
agent_data = epstein_rebellion_grid_job.results.csv_output("regular_agent_info.csv")
movement = epstein_rebellion_grid_job.results.csv_output("movement.csv")

sim_data = pd.merge(left=movement, 
         right=agent_data, on=["id","simday"], how="left").fillna(0)

sim_data.agent_status = sim_data.agent_status.map({0:"Passive", 1:"Rebel", 2:"Cop", -1:"Jailed"})

sim_data["will_to_rebel"] = sim_data.grievance - sim_data.risk * sim_data.est_arrest_prob

dates = epstein_rebellion_grid_job.results.dates()
dates = dates.rename(columns={"sim_day":"simday"}).drop(columns=["run_id"])

sim_data = sim_data.merge(dates, on="simday", how="left")

tmp = sim_data[(sim_data.simday==0)&(sim_data.agent_status=="Passive")].copy()
tmp.simday = 1
sim_data = pd.concat([sim_data, tmp])

tmp = sim_data[(sim_data.simday==0)&(sim_data.agent_status=="Cop")].copy()
tmp.simday = 1
sim_data = pd.concat([sim_data, tmp])


dict = {"agent_status":["Cop", "Passive", "Rebel", "Jailed"], 
        "simday":[0]*4, "id":[0]*4, "xloc":[-100]*4, "yloc":[-1]*4, "agent_type":[2]*4}
sim_data = pd.concat([pd.DataFrame(dict), sim_data])

sim_data = sim_data.assign(size_val=2)

In [ ]:
agent_states = (sim_data[(sim_data.agent_status!="Cop")]
 .groupby(["simday", "agent_status"]).size().to_frame().reset_index()
 .rename(columns={0:"Count"})
)

fig = go.Figure()

for x in ["Passive", "Rebel", "Jailed"]:
    fig.add_trace(
        go.Scatter(
            x=agent_states[agent_states.agent_status==x].simday,
            y=agent_states[agent_states.agent_status==x].Count,
            mode="lines",
            line=go.scatter.Line(width=3),
            showlegend=True,
            name=x)
    )


fig.update_layout(
    font_family="Epistemix Label",
    yaxis_title="Count",
    xaxis_title="Sim day",
    legend_title="Agent status",
    title="Rebellion model, grid implementation",
    title_font_size=24,
    hovermode="x unified",height=450,
)

fig.show()

In [ ]:
fig = px.scatter(sim_data, 
                 x="xloc", 
                 y="yloc", color="agent_status",
                 color_discrete_sequence=[px.colors.qualitative.Plotly[0],
                                          "white",
                                          px.colors.qualitative.Plotly[1],
                                          "gray",
                                         ],
                 opacity=0.8,
                 size="size_val", size_max=6,
                 animation_frame="simday",
                 animation_group="id"
                )

fig.update_layout(height=600, xaxis_range=[0,40])
fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
  )

fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 300
fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 0

fig.show()

In [ ]:
epstein_rebellion_grid_job.delete(interactive=False)